In [68]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import random
import time
import tiktoken

# .env 파일 로드
load_dotenv()

True

In [69]:

investing_path = r'C:\STFOPorject\STFO-2\Data\Investing_Data.json'
hankyung_path = r'C:\STFOPorject\STFO-2\Data\Hankyung_Data.json'



In [70]:
df1 = pd.read_json(investing_path, orient='records', encoding='utf-8', dtype='string')
df1.head()

,news_title,news_first_upload_time,news_last_upload_time,author,news_content,news_url,news_website,note
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,<NA>,<NA>,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,<NA>,<NA>,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,<NA>,<NA>,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,<NA>,<NA>,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,<NA>,<NA>,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>


In [71]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69916 entries, 0 to 69915
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   news_title              69765 non-null  string
 1   news_first_upload_time  69765 non-null  string
 2   news_last_upload_time   0 non-null      string
 3   author                  0 non-null      string
 4   news_content            69916 non-null  string
 5   news_url                69916 non-null  string
 6   news_website            69916 non-null  string
 7   note                    0 non-null      string
dtypes: string(8)
memory usage: 4.3 MB


In [72]:
df2 = pd.read_json(hankyung_path, orient='records', encoding='utf-8', dtype='string')
df2.head()

,news_title,news_first_upload_time,news_last_upload_time,author,news_content,news_url,news_website,note
0,"해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐",2024-12-07 PM 07:06,2024-12-07 PM 07:06,양한나 기자,인도 및 신흥시장 전문 웹3 벤처캐피털 해시드 이머전트(Hashed Emergent...,https://www.hankyung.com/article/202412071413g,Hankyung,<NA>
1,비트코인 10만달러 재탈환…이더리움도 '껑충',2024-12-07 AM 11:13,2024-12-07 AM 11:13,<NA>,가상화폐 대장주 비트코인 가격이 6일(현지시간) 10만 달러선을 재탈환했다. 시가총...,https://www.hankyung.com/article/2024120709145,Hankyung,<NA>
2,비트코인 10만달러 재돌파…'이더리움의 시간' 오나,2024-12-07 AM 09:54,2024-12-07 AM 09:54,고정삼 기자,암호화폐 대장주 비트코인이 6일(현지시간) 10만달러선 재탈환에 성공했다. 시가총액...,https://www.hankyung.com/article/2024120707037,Hankyung,<NA>
3,"""비트코인 전략비축? 헛소리""…美 전 재무장관 '일침'",2024-12-06 AM 10:15,2024-12-06 AM 10:15,<NA>,로렌스 서머스 전 미국 재무부 장관이 가상화폐 비트코인을 원유처럼 전략비축하자는 주...,https://www.hankyung.com/article/2024120685065,Hankyung,<NA>
4,"비트코인, 다시 10만달러 아래로…트럼프 '환호'",2024-12-06 AM 06:35,2024-12-06 AM 06:58,<NA>,가상화폐 비트코인이 지난 4일(현지시간) 사상 처음 10만 달러를 돌파했지만 하루만...,https://www.hankyung.com/article/2024120679115,Hankyung,<NA>


In [73]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   news_title              4000 non-null   string
 1   news_first_upload_time  4000 non-null   string
 2   news_last_upload_time   4000 non-null   string
 3   author                  283 non-null    string
 4   news_content            4000 non-null   string
 5   news_url                4000 non-null   string
 6   news_website            4000 non-null   string
 7   note                    0 non-null      string
dtypes: string(8)
memory usage: 250.1 KB


In [74]:
# !pip install pyjwt

In [75]:
import pyupbit

print(pyupbit.get_tickers())

['KRW-BTC', 'KRW-ETH', 'BTC-ETH', 'BTC-XRP', 'BTC-ETC', 'BTC-CVC', 'BTC-DGB', 'BTC-SC', 'BTC-SNT', 'BTC-WAVES', 'BTC-NMR', 'BTC-XEM', 'BTC-QTUM', 'BTC-BAT', 'BTC-LSK', 'BTC-STEEM', 'BTC-DOGE', 'BTC-BNT', 'BTC-XLM', 'BTC-ARDR', 'BTC-ARK', 'BTC-STORJ', 'BTC-GRS', 'BTC-RLC', 'USDT-BTC', 'USDT-ETH', 'USDT-XRP', 'USDT-ETC', 'KRW-NEO', 'KRW-MTL', 'KRW-XRP', 'KRW-ETC', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-ADA', 'BTC-ADA', 'BTC-MANA', 'KRW-SBD', 'BTC-SBD', 'KRW-POWR', 'BTC-POWR', 'KRW-BTG', 'USDT-ADA', 'BTC-DNT', 'BTC-ZRX', 'BTC-TRX', 'BTC-TUSD', 'BTC-LRC', 'KRW-ICX', 'KRW-EOS', 'USDT-TUSD', 'KRW-TRX', 'BTC-POLYX', 'USDT-SC', 'USDT-TRX', 'KRW-SC', 'KRW-ONT', 'KRW-ZIL', 'KRW-POLYX', 'KRW-ZRX', 'KRW-LOOM', 'BTC-BCH', 'USDT-BCH', 'KRW-BCH', 'BTC-HIFI', 'BTC-LOOM', 'KRW-BAT', 'KRW-IOST', 'USDT-DGB', 'KRW-CVC', 'KRW-IQ', 'KRW-IOTA', 'BTC-RVN', 'BTC-GO', 'BTC-BOUNTY', 'BTC-ENJ', 'KRW-HIFI', 'KRW-ONG', 'K

In [76]:
df3 = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365, to="2024-01-01")
print(df3.tail())

                           open        high         low       close  \
2023-12-27 09:00:00  57032000.0  58212000.0  56349000.0  57920000.0   
2023-12-28 09:00:00  57920000.0  58468000.0  56700000.0  56907000.0   
2023-12-29 09:00:00  56904000.0  57300000.0  56000000.0  56450000.0   
2023-12-30 09:00:00  56450000.0  57150000.0  56101000.0  56639000.0   
2023-12-31 09:00:00  56639000.0  57437000.0  56452000.0  57047000.0   

                          volume         value  
2023-12-27 09:00:00  3965.842823  2.270096e+11  
2023-12-28 09:00:00  3958.816736  2.269573e+11  
2023-12-29 09:00:00  4273.270718  2.419572e+11  
2023-12-30 09:00:00  2301.724829  1.301367e+11  
2023-12-31 09:00:00  2308.383160  1.315195e+11  


In [77]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2023-01-01 09:00:00 to 2023-12-31 09:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    365 non-null    float64
 1   high    365 non-null    float64
 2   low     365 non-null    float64
 3   close   365 non-null    float64
 4   volume  365 non-null    float64
 5   value   365 non-null    float64
dtypes: float64(6)
memory usage: 20.0 KB


In [78]:
# 필요한 열만 선택하여 새로운 데이터프레임 생성
df1_1 = df1[['news_title', 'news_first_upload_time', 'news_last_upload_time', 
             'news_content', 'news_url', 'news_website']]

# 결과 확인
print(df1_1)


                                           news_title news_first_upload_time  \
0                                      XRP 가격이 10% 상승    2024-12-08 AM 01:06   
1                    스탠다드차타드 "비트코인, 내년까지 20만달러 돌파 가능"    2024-12-07 PM 08:45   
2                 "바이낸스 상장으로 9.5배 수익"…50대 노린 꼼수 사기 기승    2024-12-07 PM 06:00   
3      [코인 리포트] "프로젝트 고문이 '가상자산 차르'"…제로엑스(ZRX) 40% 급등    2024-12-07 AM 03:13   
4                       폴 앳킨스 SEC 위원장 지명 소식에 업계 "대환영"    2024-12-07 AM 01:22   
...                                               ...                    ...   
69911                바이낸스 CEO "바이낸스, 마진 거래 서비스 출시 예정"    2019-05-08 PM 06:00   
69912                 유명 암호화폐 트레이더 "비트파이넥스 LEO 토큰 주의"    2019-05-08 PM 05:57   
69913                   日 거래소 비트플라이어, 사용자 자산 긴급 점검 진행    2019-05-08 PM 05:54   
69914        ETH 코어 개발자 "블록 리오그 여부, 바이낸스 아닌 커뮤니티의 결정"    2019-05-08 PM 05:33   
69915                   페이팔 CFO "블록체인·암호화폐, 아직은 먼 미래"    2019-05-08 PM 05:19   

      news_last_upload_time  \
0       

In [79]:
# 필요한 열만 선택하여 새로운 데이터프레임 생성
df2_1 = df2[['news_title', 'news_first_upload_time', 'news_last_upload_time',  
             'news_content', 'news_url', 'news_website']]

# 결과 확인
print(df2_1) 

                               news_title news_first_upload_time  \
0      해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐    2024-12-07 PM 07:06   
1               비트코인 10만달러 재탈환…이더리움도 '껑충'    2024-12-07 AM 11:13   
2            비트코인 10만달러 재돌파…'이더리움의 시간' 오나    2024-12-07 AM 09:54   
3          "비트코인 전략비축? 헛소리"…美 전 재무장관 '일침'    2024-12-06 AM 10:15   
4             비트코인, 다시 10만달러 아래로…트럼프 '환호'    2024-12-06 AM 06:35   
...                                   ...                    ...   
3995  조은희 "서울시, 박원순 '대북코인' 연루 의혹 진실 규명해야"    2022-10-12 AM 11:19   
3996              구글 검색창에 이더리움 지갑 잔액 표시된다    2022-10-11 PM 11:00   
3997          美 대형 은행 BNY멜론, 가상자산 커스터디 지원    2022-10-11 PM 09:49   
3998           구글 클라우드, 내년 초 가상자산 결제 도입한다    2022-10-11 PM 09:28   
3999         영란은행, 채권 시장 추가 개입…물가연동채권도 매입    2022-10-11 PM 04:13   

     news_last_upload_time                                       news_content  \
0      2024-12-07 PM 07:06  인도 및 신흥시장 전문 웹3 벤처캐피털 해시드 이머전트(Hashed Emergent...   
1      2024-12-07 AM 

In [80]:
# 행 단위로 결합
df_combined = pd.concat([df1_1, df2_1], ignore_index=True)

df_combined

,news_title,news_first_upload_time,news_last_upload_time,news_content,news_url,news_website
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,<NA>,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,<NA>,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,<NA>,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,<NA>,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,<NA>,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
...,...,...,...,...,...,...
73911,"조은희 ""서울시, 박원순 '대북코인' 연루 의혹 진실 규명해야""",2022-10-12 AM 11:19,2022-10-12 AM 11:20,조은희 국민의힘 의원이 12일 서울시 국정감사에서 박원순 전 서울시장의 '대북 코인...,https://www.hankyung.com/article/202210122014B,Hankyung
73912,구글 검색창에 이더리움 지갑 잔액 표시된다,2022-10-11 PM 11:00,2022-10-11 PM 11:01,구글 검색창에 이더리움(ETH) 주소를 검색하면 지갑의 이더리움 잔액이 표시된다. ...,https://www.hankyung.com/article/202210110754B,Hankyung
73913,"美 대형 은행 BNY멜론, 가상자산 커스터디 지원",2022-10-11 PM 09:49,2022-10-11 PM 09:51,미국에서 가장 오래된 은행 BNY멜론(BNY Mellon)이 가상자산(암호화폐) 커...,https://www.hankyung.com/article/202210110661B,Hankyung
73914,"구글 클라우드, 내년 초 가상자산 결제 도입한다",2022-10-11 PM 09:28,2022-10-11 PM 09:28,구글 클라우드가 내년 초 가상자산(암호화폐) 결제를 도입할 예정이라고 밝혔다. 11...,https://www.hankyung.com/article/202210110619B,Hankyung


In [81]:
# 내림차순 정렬
df_sorted = df_combined.sort_values(by="news_first_upload_time", ascending=False)


In [82]:
df_sorted

,news_title,news_first_upload_time,news_last_upload_time,news_content,news_url,news_website
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,<NA>,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,<NA>,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing
69916,"해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐",2024-12-07 PM 07:06,2024-12-07 PM 07:06,인도 및 신흥시장 전문 웹3 벤처캐피털 해시드 이머전트(Hashed Emergent...,https://www.hankyung.com/article/202412071413g,Hankyung
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,<NA>,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing
69917,비트코인 10만달러 재탈환…이더리움도 '껑충',2024-12-07 AM 11:13,2024-12-07 AM 11:13,가상화폐 대장주 비트코인 가격이 6일(현지시간) 10만 달러선을 재탈환했다. 시가총...,https://www.hankyung.com/article/2024120709145,Hankyung
...,...,...,...,...,...,...
66311,<NA>,<NA>,<NA>,,https://kr.investing.com/news/cryptocurrency-n...,Investing
68774,<NA>,<NA>,<NA>,,https://kr.investing.com/news/cryptocurrency-n...,Investing
68919,<NA>,<NA>,<NA>,,https://kr.investing.com/news/cryptocurrency-n...,Investing
69341,<NA>,<NA>,<NA>,,https://kr.investing.com/news/cryptocurrency-n...,Investing


In [83]:
df_sorted.isnull().sum()

news_title                  151
news_first_upload_time      151
news_last_upload_time     69916
news_content                  0
news_url                      0
news_website                  0
dtype: int64

In [84]:
# news_title 열에 NaN 값이 있는 행 삭제
df_sorted_cleaned = df_sorted.dropna(subset=["news_title"])

# 결과 확인
print(df_sorted_cleaned)


                                     news_title news_first_upload_time  \
0                                XRP 가격이 10% 상승    2024-12-08 AM 01:06   
1              스탠다드차타드 "비트코인, 내년까지 20만달러 돌파 가능"    2024-12-07 PM 08:45   
69916        해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐    2024-12-07 PM 07:06   
2           "바이낸스 상장으로 9.5배 수익"…50대 노린 꼼수 사기 기승    2024-12-07 PM 06:00   
69917                 비트코인 10만달러 재탈환…이더리움도 '껑충'    2024-12-07 AM 11:13   
...                                         ...                    ...   
69911          바이낸스 CEO "바이낸스, 마진 거래 서비스 출시 예정"    2019-05-08 PM 06:00   
69912           유명 암호화폐 트레이더 "비트파이넥스 LEO 토큰 주의"    2019-05-08 PM 05:57   
69913             日 거래소 비트플라이어, 사용자 자산 긴급 점검 진행    2019-05-08 PM 05:54   
69914  ETH 코어 개발자 "블록 리오그 여부, 바이낸스 아닌 커뮤니티의 결정"    2019-05-08 PM 05:33   
69915             페이팔 CFO "블록체인·암호화폐, 아직은 먼 미래"    2019-05-08 PM 05:19   

      news_last_upload_time  \
0                      <NA>   
1                      <NA>   
69916   2024-12-07

In [85]:
df_sorted_cleaned.isnull().sum()

news_title                    0
news_first_upload_time        0
news_last_upload_time     69765
news_content                  0
news_url                      0
news_website                  0
dtype: int64

In [86]:

df_sorted_100 = df_sorted_cleaned[:100]
df_sorted_100.iloc[99]

news_title                                 "여권으로 인증하세요"…월드ID 3.0 파일럿 서비스 시작
news_first_upload_time                                  2024-11-29 PM 11:25
news_last_upload_time                                                  <NA>
news_content              월드ID에서 여권으로 자격증명을 할 수 있게 된다.
29일 툴스 포 휴머니티(TFH...
news_url                  https://kr.investing.com/news/cryptocurrency-n...
news_website                                                      Investing
Name: 83, dtype: string

In [87]:
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API Key loaded successfully!")
else:
    print("API Key not found!")

API Key loaded successfully!


In [88]:
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

In [89]:
def get_completion(prompt, model = 'gpt-4o'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [90]:
prompt = f"""
너는 신문기사를 분석하는 로봇이다. 내가 신문기사 100개가 담긴 데이터 프레임을 입력할 것이다. 
데이터 프레임을 바탕으로 나의 질문에 답하여라.



<데이터 프레임> : <{df_sorted_100}>

<질문>
계엄령이 발생한 날은 언제인가? 관련 신문기사 내용을 검색한 후 알려줘


"""
response = get_completion(prompt)
print(response)

주어진 데이터 프레임에서 계엄령과 관련된 기사를 찾기 위해서는 `news_title` 또는 `news_content` 열에서 "계엄령"이라는 키워드를 검색해야 합니다. 그러나 제공된 데이터 프레임의 일부만 보았을 때, "계엄령"이라는 단어가 포함된 기사는 보이지 않습니다. 

계엄령과 관련된 기사가 데이터 프레임에 포함되어 있는지 확인하려면, 전체 데이터 프레임에서 "계엄령"이라는 키워드를 검색해야 합니다. 만약 해당 키워드가 포함된 기사가 있다면, 그 기사의 `news_first_upload_time` 또는 `news_last_upload_time`을 통해 계엄령이 발생한 날짜를 알 수 있을 것입니다. 

데이터 프레임 전체를 검색할 수 있는 방법이 제공되지 않은 상황에서는, 계엄령과 관련된 기사가 포함되어 있는지 여부를 확정할 수 없습니다. 데이터 프레임 전체를 검색할 수 있는 환경에서 "계엄령"이라는 키워드를 검색해 보시기 바랍니다.


In [91]:
df_sorted_100.loc[6, :]


news_title                                   계엄령 발표에 업비트·빗썸 '먹통'…관리체계 강화 시급
news_first_upload_time                                  2024-12-06 PM 08:43
news_last_upload_time                                                  <NA>
news_content              업비트·빗썸 등 국내 주요 가상자산 거래소가 지난 3일 비상 계엄령 선포 이후 일제...
news_url                  https://kr.investing.com/news/cryptocurrency-n...
news_website                                                      Investing
Name: 6, dtype: string

In [99]:
df_sorted_100[df_sorted_100['news_content'].str.contains('계엄')]

,news_title,news_first_upload_time,news_last_upload_time,news_content,news_url,news_website
6,계엄령 발표에 업비트·빗썸 '먹통'…관리체계 강화 시급,2024-12-06 PM 08:43,<NA>,업비트·빗썸 등 국내 주요 가상자산 거래소가 지난 3일 비상 계엄령 선포 이후 일제...,https://kr.investing.com/news/cryptocurrency-n...,Investing
15,"7080대도 북적… 빗썸 라운지, 비트코인 고공행진에 '후끈'",2024-12-06 AM 01:42,<NA>,"“코인이 미래라고 하길래 이제부터 투자를 시작해보려고요."" 지난 4일 서울 서초구 ...",https://kr.investing.com/news/cryptocurrency-n...,Investing
29,"계엄 담화에 급락한 코인, 외국 투자자들도 '줍줍' 노렸다",2024-12-05 PM 02:30,<NA>,윤석열 대통령의 느닷없는 비상계엄 선언으로 3일 늦은 밤 국내 주요 가상자산 가격이...,https://kr.investing.com/news/cryptocurrency-n...,Investing
69922,"""계엄령 수혜 코인 등장""…비상계엄 사태에 111% 폭등",2024-12-05 AM 11:55,2024-12-05 AM 11:55,"윤석열 대통령의 비상계엄령의 수혜 코인이 등장했다. 국내 원화거래소 업비트 기준, ...",https://www.hankyung.com/article/2024120562565,Hankyung
31,계엄령發 비트코인 폭락에도 글로벌 시세 '평온'…韓 영향력 약화,2024-12-05 AM 03:52,<NA>,전날 비상계엄 선포로 인한 비트코인(BTC) 폭락세가 국내 거래소에만 국한되면서 한...,https://kr.investing.com/news/cryptocurrency-n...,Investing
32,가상자산 급등락…‘역 김치프리미엄’ 노린 해외 자금 습격[비상계엄 후폭풍],2024-12-05 AM 02:53,<NA>,윤석열 대통령의 비상계엄 선언 직후 국내 가상자산 거래소 업비트에서 가상자산 가격이...,https://kr.investing.com/news/cryptocurrency-n...,Investing
34,"'계엄령 최대 수혜자' 업비트·빗썸, 하루에 수백억 벌어…국내 증시 압도",2024-12-05 AM 01:39,<NA>,윤석열 대통령의 비상계엄령 선포 이후 국내 주요 가상자산 거래소 업비트·빗썸의 거래...,https://kr.investing.com/news/cryptocurrency-n...,Investing
35,"비트코인, 계엄 해제 반영에 1억3500만원대 회복···한때 ‘역김치프리미엄’ 30...",2024-12-05 AM 01:15,<NA>,투데이코리아 - ▲ 4일 서울 서초구 빗썸라운지 전광판에 비트코인 등 가상화폐 시세...,https://kr.investing.com/news/cryptocurrency-n...,Investing
37,[점심브리핑] 계엄령 선포에 업비트로 2300억원 스테이블코인 몰려…'저가 매수 기회',2024-12-04 PM 11:53,<NA>,주요 가상자산이 상승세다. 4일 오후 1시 빗썸 기준 비트코인(BTC)은 전일 대비...,https://kr.investing.com/news/cryptocurrency-n...,Investing
39,가상자산 시장 '계엄령 쇼크' 하루만에 진정…비트코인 1억 3000만원대,2024-12-04 PM 07:04,<NA>,윤석열 대통령의 비상계엄령 선포로 출렁였던 가상자산 가격이 안정세를 되찾고 있다. ...,https://kr.investing.com/news/cryptocurrency-n...,Investing


In [98]:
len(df_sorted_100[df_sorted_100['news_content'].str.contains('계엄')])


17

In [93]:
prompt = f"""
너는 신문기사를 분석하는 로봇이다. 내가 신문기사 담긴 데이터 프레임을 입력할 것이다. 
데이터 프레임을 바탕으로 나의 질문에 답하여라.



<데이터 프레임> : <{df_sorted_100.iloc[99]}>

<질문>
데이터 프레임에는 어떤 정보들이 있는지 알려줘


"""
response = get_completion(prompt)
print(response)

데이터 프레임에는 다음과 같은 정보들이 포함되어 있습니다:

1. **news_title**: 기사 제목입니다. 예시로 "여권으로 인증하세요"…월드ID 3.0 파일럿 서비스 시작"이라는 제목이 있습니다.

2. **news_first_upload_time**: 기사가 처음 업로드된 시간입니다. 예시로 "2024-11-29 PM 11:25"라는 시간이 있습니다.

3. **news_last_upload_time**: 기사가 마지막으로 업로드된 시간입니다. 이 경우에는 정보가 제공되지 않아 `<NA>`로 표시되어 있습니다.

4. **news_content**: 기사 내용입니다. 예시로 "월드ID에서 여권으로 자격증명을 할 수 있게 된다. 29일 툴스 포 휴머니티(TFH..."라는 내용이 포함되어 있습니다.

5. **news_url**: 기사의 URL입니다. 예시로 "https://kr.investing.com/news/cryptocurrency-n..."라는 URL이 있습니다.

6. **news_website**: 기사가 게시된 웹사이트의 이름입니다. 예시로 "Investing"이라는 웹사이트가 있습니다.


In [94]:
df_sorted_100.iloc[99]

news_title                                 "여권으로 인증하세요"…월드ID 3.0 파일럿 서비스 시작
news_first_upload_time                                  2024-11-29 PM 11:25
news_last_upload_time                                                  <NA>
news_content              월드ID에서 여권으로 자격증명을 할 수 있게 된다.
29일 툴스 포 휴머니티(TFH...
news_url                  https://kr.investing.com/news/cryptocurrency-n...
news_website                                                      Investing
Name: 83, dtype: string

In [95]:
prompt = f"""
너는 코인데이터를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 



<코인 데이터> : <{df3}>

<질문>
전체 자료 중, 최고가와 최저가에 대해 알려줘


"""
response = get_completion(prompt)
print(response)

주어진 코인 데이터에서 최고가와 최저가를 찾기 위해서는 'high' 열과 'low' 열을 각각 확인해야 합니다.

- 최고가 (high): 2023년 데이터에서 최고가는 2023-12-28에 기록된 58,468,000.0입니다.
- 최저가 (low): 2023년 데이터에서 최저가는 2023-01-01에 기록된 21,002,000.0입니다.

이 값들은 주어진 데이터의 'high'와 'low' 열에서 각각 가장 높은 값과 가장 낮은 값을 나타냅니다.


In [100]:
# 보고서 형태의 챗봇

prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 공신력있는 보고서 형태로 작성해줘


<신문 기사> : <{df_sorted_100[df_sorted_100['news_content'].str.contains('계엄')]}>
<코인 데이터> : <{df3}>

<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 '계엄'활동이 코인 가격에 어떤 충격을 줬는지 알려줘.


"""
response = get_completion(prompt)
print(response)

# 보고서: 계엄령이 가상자산 시장에 미친 영향 분석

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   - 2024년 12월 3일과 4일 사이의 데이터는 제공되지 않았으나, 신문 기사에 따르면 비트코인 가격은 계엄령 발표 직후 급락한 후 빠르게 회복되었습니다.
   - 2023년 12월 말의 데이터와 비교했을 때, 계엄령 발표 전후의 가격 변동은 상당한 수준이었음을 추정할 수 있습니다.

2. **거래량과 가치**:
   - 계엄령 발표 직후 거래소의 마비로 인해 거래량이 일시적으로 감소했을 가능성이 있습니다. 그러나 이후 저가 매수 기회로 인식되면서 거래량이 증가했을 것으로 보입니다.

## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급

In [ ]:
# 보고서 형태의 챗봇

prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. <형식>의 내용에 맞게 작성해줘.

<신문 기사> : <{df_sorted_100[df_sorted_100['news_content'].str.contains('계엄')]}>

<코인 데이터> : <{df3}>

<형식>
# 2024년 보고서: 계엄령이 가상자산 시장에 미친 영향 분석

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   - 2024년 12월 3일과 4일 사이의 데이터는 제공되지 않았으나, 신문 기사에 따르면 비트코인 가격은 계엄령 발표 직후 급락한 후 빠르게 회복되었습니다.
   - 2023년 12월 말의 데이터와 비교했을 때, 계엄령 발표 전후의 가격 변동은 상당한 수준이었음을 추정할 수 있습니다.

2. **거래량과 가치**:
   - 계엄령 발표 직후 거래소의 마비로 인해 거래량이 일시적으로 감소했을 가능성이 있습니다. 그러나 이후 저가 매수 기회로 인식되면서 거래량이 증가했을 것으로 보입니다.

## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급락은 외국 투자자들에게 저가 매수 기회를 제공하였고, 이는 역 김치프리미엄 현상을 초래했습니다. 전반적으로, 계엄령은 가상자산 시장의 변동성을 일시적으로 증가시켰으나, 시장의 회복력과 안정성을 확인할 수 있는 계기가 되었습니다.

## 참고 자료
- 신문 기사: Investing, Hankyung
- 코인 데이터: 2023년 1월 1일부터 12월 31일까지의 비트코인 가격 및 거래량 데이터

본 보고서는 계엄령이 가상자산 시장에 미친 영향을 종합적으로 분석하였으며, 향후 유사한 정치적 사건이 발생할 경우 시장의 반응을 예측하는 데 유용한 자료로 활용될 수 있습니다.






<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 '계엄'활동이 코인 가격에 어떤 충격을 줬는지 알려줘.


"""
response = get_completion(prompt)
print(response)